In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import paraguayfloodspy.visualize as viz # custom plotting library

Define parameters

In [ ]:
%matplotlib inline
savefig = True

Read in data

In [ ]:
wt = pd.read_csv('../data/derived/WeatherTypes.csv', parse_dates=True, index_col='time')
mjo = pd.read_csv('../data/accessed/daily_indices.csv', parse_dates=True, index_col='time')
enso = pd.read_csv('../data/accessed/monthly_indices.csv', parse_dates=True, index_col='time')
enso = enso.resample('1D').ffill()
mrg = wt.join(mjo).join(enso)
mrg.loc[mrg['amplitude'] < 1, 'phase'] =  0
mrg['enso_state'] = 0
mrg.loc[mrg['nino_34'] < -1, 'enso_state'] = -1
mrg.loc[mrg['nino_34'] > 1, 'enso_state'] = 1
mrg.head()

In [ ]:
conditional = mrg.groupby(['phase', 'enso_state'])['wtype'].apply(lambda g: g.value_counts() / g.size)
conditional = conditional.to_xarray()
anomaly = conditional - conditional.mean(dim=['level_2'])

In [ ]:
n_sim = 3000
anomalies = []
for i in np.arange(n_sim):
    wt_seq = np.random.choice(mrg['wtype'], size=mrg.shape[0], replace=True)
    simulated = pd.DataFrame({'wtype': wt_seq, 'phase': mrg['phase'], 'enso_state': mrg['enso_state']})
    conditional = simulated.groupby(['phase', 'enso_state'])['wtype'].apply(lambda g: g.value_counts() / g.size)
    conditional = conditional.to_xarray()
    anomalies.append(conditional - conditional.mean(dim=['level_2']))
anomalies = xr.concat(anomalies, dim='temp')

Select significant values

In [ ]:
alpha = 0.10
upper = anomalies.quantile(1-alpha/2, dim='temp')
lower = anomalies.quantile(alpha/2, dim='temp')
significant = np.logical_or((anomaly >= upper), (anomaly <= lower))

Plot

In [ ]:
enso_states = {-1: 'La Niña', 0: 'Neutral', 1: 'El Niño'}

In [ ]:
p = anomaly.where(significant).plot(
    x='level_2', y='phase', col='enso_state', 
    cmap='PuOr',
    figsize=(12, 5)
)
for i,es in enumerate(enso_states):
    ax = p.axes.flat[i]
    ax.set_title(enso_states.get(i-1, ''))
    ax.set_xlabel('Weather Type')
p.axes[0,0].set_ylabel('MJO Phase')
p.cbar.set_label('Anomalous Probability of Occurrence', rotation=270)
if savefig:
    plt.savefig('../figs/WT_MJO_ENSO.pdf', bbox_inches='tight')